In [1]:
from tool import *
import sys

# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

## PO DB 업데이트

In [2]:
filepath = 'D:/Downloads/' + [file for file in os.listdir('D:/Downloads/') if 'poshThru' in file][-1] # 파일이름에 poshThru가 들어가는 파일 중에 가장 최신 파일을 선택함
df = pd.read_excel(filepath, sheet_name='DataSet').iloc[:, :48]

if df['Model'][0].split('.')[0] == 'PH30N':
    vendor = 'Wanlida'
else:
    vendor = df['Model'].apply(lambda x:x.split('-')[0]).replace(srt_model).replace(vendor_find)[0]

if vendor not in ['Quanta', 'Pegatron', 'Wanlida', 'Wingtech']:
    sys.exit()

df = df[['Model', 'PO No.', 'Ship To', 'Shipping', 'Cancel', 'PO', 'OQC Report', 'OQC Date', 'OQC Result', 'Ship', 'Issued Date', 'RSD','Ship Date', 'BL No', 'BL Status', 'Method', 'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.', 'Final Destination']]
df.loc[df['PO1 No'].notnull(), 'PO1 No'] = df.loc[df['PO1 No'].notnull(),'PO1 No'].astype('int').astype('str')
df.loc[df['SO No.'].notnull(), 'SO No.'] = df.loc[df['SO No.'].notnull(), 'SO No.'].astype('int').astype('str')
df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']] = df[['OQC Date', 'PO1 No', 'SO No.', 'Final Destination']].fillna('-')

df['Ship'].fillna(0, inplace=True)
df['Ship'] = df['Ship'].astype('int')
df['Issued Date'] = pd.to_datetime(df['Issued Date'])
df['PO Week'] = df['Issued Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['PO Month'] = df['Issued Date'].apply(get_month_from_date)
df['PO Year'] = df['Issued Date'].dt.isocalendar().year
df['RSD'] = pd.to_datetime(df['RSD']) # PO의 RSD를 Datetime 형식으로 변경
df['RSD Week'] = df['RSD'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['RSD Month'] = df['RSD'].apply(get_month_from_date)
df['RSD Year'] = df['RSD'].dt.isocalendar().year
df['RSD Week Year'] = df['RSD'].apply(getfirstdate_year)
df['RSD Week Month'] = df['RSD'].apply(getfirstdate_month)
df['RSD Week Day'] = df['RSD'].apply(getfirstdate_day)
df['RSD Week Name'] = df['RSD Week Year'].astype(str) + '-' + df['RSD Week Month'].astype(str) + '-' + df['RSD Week Day'].astype(str) + '(W' + df['RSD Week'].astype(str) +')'
df.rename(columns={'Model':'Mapping Model.Suffix'}, inplace=True)

if vendor != 'Wanlida':
    df['Model'] = df['Mapping Model.Suffix'].apply(lambda x: x.split('-')[0])
    df['Series'] = df['Model'].replace(srt_model)

df['Ship To'] = df['Ship To'].replace(site_name_adjust_map)
df['Country'] = df['Ship To'].replace(site_map)
df['Region'] = df['Country'].replace(country_map)

if vendor != 'Wanlida':
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country', 'Series', 'Model',
                        'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                        'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                        'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')
else:
    df1 = df.groupby(['PO No.', 'Cancel', 'Issued Date', 'PO Year', 'PO Month', 'PO Week', 'Region', 'Country',
                    'Mapping Model.Suffix', 'Ship To', 'RSD', 'RSD Year', 'RSD Month', 'RSD Week', 'RSD Week Name', 'Method',
                    'Price Term', 'Unit Price', 'Currency', 'Payment Term', 'PO1 No', 'SO No.',
                    'Final Destination'])[['PO']].sum().reset_index().set_index('PO No.')  

with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO_df = pickle.load(f)

In [3]:
PO_df1 = PO_df.copy()

In [4]:
prev_po = PO_df.shape[0]

In [5]:
for i in df1.index:
    PO_df.loc[i, :] = df1.loc[i, :]

In [6]:
updated_po = PO_df.shape[0]

In [7]:
with open(f'D:/Data/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)
# DB backup 폴더에도 저장
with open(f'D:/Data/DB backup/{vendor} PO DB.bin', 'wb') as f:
    pickle.dump(PO_df, f)

## Shipment result DB 업데이트

In [8]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
     SR_df = pickle.load(f)

df = df[df['Ship Date'].notnull()]
df.loc[:, 'Ship Date'] = pd.to_datetime(df.loc[:, 'Ship Date'])
df['Ship Week'] = df['Ship Date'].dt.isocalendar().week.apply(lambda x:'{0:02d}'.format(x))
df['Ship Month'] = df['Ship Date'].apply(get_month_from_date)
df['Ship Year'] = df['Ship Date'].dt.isocalendar().year
df['Ship Week month'] = df['Ship Date'].apply(getfirstdate_month)
df['Ship Week day'] = df['Ship Date'].apply(getfirstdate_day)
df['Ship Week year'] = df['Ship Date'].apply(getfirstdate_year)
df['Ship Week year'] = df['Ship Week year'].astype(str)
df['Ship Week month'] = df['Ship Week month'].astype(str)
df['Ship Week day'] = df['Ship Week day'].astype(str)
df['Ship Week'] = df['Ship Week'].astype(str)
df['Week Name'] = df['Ship Week year'] + '-' + df['Ship Week month'] + '-' + df['Ship Week day'] + '(W' + df['Ship Week'] + ')'
df = df.set_index(['PO No.', 'BL No'])

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_6748\37446698.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'Ship Date'] = pd.to_datetime(df.loc[:, 'Ship Date'])


In [9]:
SR_df1 = SR_df.copy()

In [10]:
prev_sr = SR_df.shape[0]

In [11]:
for i in df.index:
    SR_df.loc[i, :] = df.loc[i, :]

In [12]:
updated_sr = SR_df.shape[0]

In [13]:
# PO 대비 선적수량이 더 많은 경우가 없는지 check
with open(f'D:/Data/{vendor} PO DB.bin', 'rb') as f:
    PO = pickle.load(f)
with open(f'D:/Data/{vendor} shipment result DB.bin', 'rb') as f:
    SR = pickle.load(f)

PO = PO[PO['Cancel']=='N']
df = pd.concat([PO['PO'], SR.groupby('PO No.')['Ship'].sum()], axis=1)
df.fillna(0, inplace=True)
df['Open'] = df['PO'] - df['Ship']
df = df[df['Open'] < 0]

In [14]:
if df.shape[0] != 0:
    print(df)
    sys.exit()

In [15]:
with open(f'D:/Data/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)

In [16]:
with open(f'D:/Data/DB backup/{vendor} shipment result DB.bin', 'wb') as f:
    pickle.dump(SR_df, f)
print(f"{vendor}의 DB를 업데이트 하였습니다. PO는 {updated_po - prev_po} 건, 선적은 {updated_sr - prev_sr} 건 추가 되었습니다. ")
print(filepath)

Wingtech의 DB를 업데이트 하였습니다. PO는 0 건, 선적은 0 건 추가 되었습니다. 
D:/Downloads/poshThru_20230424082459.xls


In [17]:
PO_df.drop(index=PO_df1.index) # 추가된 PO 리스트

,Cancel,Issued Date,PO Year,PO Month,PO Week,Region,Country,Series,Model,Mapping Model.Suffix,Ship To,RSD,RSD Year,RSD Month,RSD Week,RSD Week Name,Method,Price Term,Unit Price,Currency,Payment Term,PO1 No,SO No.,Final Destination,PO
PO No.,,,,,,,,,,,,,,,,,,,,,,,,,


In [18]:
SR_df.drop(index=SR_df1.index) # 추가된 선적실적

,,Mapping Model.Suffix,Ship To,Shipping,Cancel,PO,OQC Report,OQC Date,OQC Result,Ship,Issued Date,RSD,Ship Date,BL Status,Method,Price Term,Unit Price,Currency,Payment Term,PO1 No,SO No.,Final Destination,PO Week,PO Month,PO Year,RSD Week,RSD Month,RSD Year,RSD Week Year,RSD Week Month,RSD Week Day,RSD Week Name,Model,Series,Country,Region,Ship Week,Ship Month,Ship Year,Ship Week month,Ship Week day,Ship Week year,Week Name
PO No.,BL No,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
